In [ ]:
!pip install crewai crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq PyGithub
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.8.0 which is incompatible.


In [ ]:
import os
import google.generativeai as genai
from crewai import Agent, Task, Crew, Process
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional

# Set API key for Gemini
os.environ["GOOGLE_API_KEY"] = ""  # Replace with your actual API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Define Gemini LLM Model
# Use 'models/gemini-pro-vision' instead of 'gemini-pro'
model = genai.GenerativeModel('models/gemini-1.5-flash')

# Create a custom LLM class for Gemini
class GeminiLLM(LLM):
    model: Any

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.model.generate_content(prompt)
        return response.text

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": "models/gemini-pro-vision"} # Updated model name here

    @property
    def _llm_type(self) -> str:
        return "gemini"

gemini_llm = GeminiLLM(model=model)

# Define the Planning Agent
planning_agent = Agent(
    llm=gemini_llm,
    role="Planning Agent",
    goal="Develop the book's concept, outline, characters, and world.",
    backstory="An experienced author specializing in planning and structuring novels.",
    allow_delegation=False,
    verbose=True
)

# Define the Writing Agent
writing_agent = Agent(
    llm=gemini_llm,
    role="Writing Agent",
    goal="Write detailed chapters based on the provided outline and character details.",
    backstory="A creative writer adept at bringing stories to life.",
    allow_delegation=False,
    verbose=True
)

# Define the Editing Agent
editing_agent = Agent(
    llm=gemini_llm,
    role="Editing Agent",
    goal="Edit the written chapters for clarity, coherence, and grammatical accuracy.",
    backstory="A meticulous editor with an eye for detail.",
    allow_delegation=False,
    verbose=True
)

# Define the Fact-Checking Agent
fact_checking_agent = Agent(
    llm=gemini_llm,
    role="Fact-Checking Agent",
    goal="Verify the accuracy of all factual information presented in the book.",
    backstory="A diligent researcher ensuring all facts are correct.",
    allow_delegation=False,
    verbose=True
)

# Define the Publishing Agent
publishing_agent = Agent(
    llm=gemini_llm,
    role="Publishing Agent",
    goal="Format the manuscript and prepare it for publication.",
    backstory="An expert in publishing standards and formatting.",
    allow_delegation=False,
    verbose=True
)

# Define the tasks for each agent
tasks = [
    Task(
        description="Develop the book's concept, outline, characters, and world.",
        expected_output="A comprehensive plan including theme, genre, outline, character profiles, and world details.",
        agent=planning_agent
    ),
    Task(
        description="Write detailed chapters based on the provided outline and character details. Each chapter should be at least 1000 words.",
        expected_output="Drafts of all chapters in the book.",
        agent=writing_agent
    ),
    Task(
        description="Edit the written chapters for clarity, coherence, and grammatical accuracy.",
        expected_output="Edited versions of all chapters.",
        agent=editing_agent
    ),
    Task(
        description="Verify the accuracy of all factual information presented in the book.",
        expected_output="A report confirming the accuracy of all facts or detailing necessary corrections.",
        agent=fact_checking_agent
    ),
    Task(
        description="Format the manuscript and prepare it for publication.",
        expected_output="A finalized manuscript ready for publication.",
        agent=publishing_agent
    )
]

# Assemble the crew
book_writing_crew = Crew(
    agents=[planning_agent, writing_agent, editing_agent, fact_checking_agent, publishing_agent],
    tasks=tasks,
    process=Process.sequential,
    verbose=True
)

# Execute the workflow
if __name__ == "__main__":
    result = book_writing_crew.kickoff()
    print("Final Manuscript:", result)

 [2025-03-16 09:59:37][DEBUG]: == Working Agent: Planning Agent
 [2025-03-16 09:59:37][INFO]: == Starting Task: Develop the book's concept, outline, characters, and world.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:

**Book Title:**  The Clockwork Heart of Aethelred

**Genre:**  Steampunk Fantasy

**Theme:** The dangers of unchecked ambition and the redemptive power of compassion, juxtaposed against a backdrop of societal upheaval and technological advancement.

**Logline:** In a world powered by intricate clockwork mechanisms, a brilliant but ruthless inventor races to complete his ultimate creation – a machine capable of controlling time itself – while a young clockwork mechanic fights to stop him and save her fractured city.


**World Details:**

* **Aethelred:** A sprawling metropolis built on the ruins of a forgotten empire.  It's a city of towering clockwork structures, intricate gears whirring constantly, steam hissing from v

In [ ]:
import os
import re
import base64
import requests
from github import Github
import google.generativeai as genai
from crewai import Agent, Task, Crew, Process
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional

# ===================================================
# Environment & GitHub Configuration (update as needed)
# ===================================================
os.environ["GOOGLE_API_KEY"] = ""  # Replace with your Gemini API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('models/gemini-1.5-flash')

GITHUB_TOKEN = "" # Replace with your GitHub token
GITHUB_USERNAME = "mannoz369"
REPO_OWNER = "mannoz369"
REPO_NAME_PART = "AI-Book-Writing-Agent"
REPO_FULL_NAME = f"{REPO_OWNER}/{REPO_NAME_PART}"
BRANCH = "main"
BASE_URL = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME_PART}/contents"

# ===================================================
# Define Gemini LLM Class
# ===================================================
class GeminiLLM(LLM):
    model: Any

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.model.generate_content(prompt)
        return response.text

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": "gemini-pro"}

    @property
    def _llm_type(self) -> str:
        return "gemini"

gemini_llm = GeminiLLM(model=model)

# ===================================================
# Define CrewAI Agents
# ===================================================
planning_agent = Agent(
    llm=gemini_llm,
    role="Planning Agent",
    goal="Develop the book's concept, outline, characters, world details, and generate a list of 10 chapter titles in a proper markdown format for making it readable. Use all markdown format structuring for heading and paragraphs to make it readable",
    backstory="An experienced author specializing in planning and structuring novels.",
    verbose=True
)

writing_agent = Agent(
    llm=gemini_llm,
    role="Writing Agent",
    goal="Write detailed chapters based on the provided outline and chapter titles. Each chapter should be at least 1000 words. Use all markdown format structuring for heading and paragraphs to make it readable",
    backstory="A creative writer adept at bringing stories to life.",
    verbose=True
)

formatting_agent = Agent(
    llm=gemini_llm,
    role="Markdown Formatting Agent",
    goal="Format and structure the text into a well-structured Markdown document with proper headings, spacing, and readability improvements.",
    backstory="A skilled editor who ensures content is properly formatted for GitHub Markdown preview.",
    verbose=True
)

# ===================================================
# Define CrewAI Tasks (With expected_output)
# ===================================================
plan_book_task = Task(
    description="Develop a book concept, outline, characters, and generate 10 chapter title in a structured way. Use all markdown format structuring for heading and paragraphs to make it readable",
    agent=planning_agent,
    expected_output="A structured book outline with 10 clearly numbered chapter titles in a proper structured Markdown format. Use all markdown format structuring for heading and paragraphs to make it readable"
)

write_chapter_task = Task(
    description="Write a detailed chapter based on the given title and previous chapters' context.",
    agent=writing_agent,
    expected_output="A well-structured 1000-word chapter continuing the story."
)

format_markdown_task = Task(
    description="Format the chapter content to ensure correct Markdown headings, paragraphs, lists, and readability improvements.",
    agent=formatting_agent,
    expected_output="A fully formatted Markdown document with improved structure and readability."
)

# ===================================================
# GitHub Functions (Same as before)
# ===================================================
def github_commit(repo_name, file_content, filename, commit_message):
    try:
        g = Github(GITHUB_TOKEN)
        repo = g.get_repo(repo_name)
        try:
            contents = repo.get_contents(filename, ref=BRANCH)
            repo.update_file(contents.path, commit_message, file_content, contents.sha, branch=BRANCH)
            print(f"Updated {filename} successfully.")
        except:
            repo.create_file(filename, commit_message, file_content, branch=BRANCH)
            print(f"Created {filename} successfully.")
    except Exception as e:
        print(f"Error committing {filename}: {e}")

def list_files(path=""):
    url = f"{BASE_URL}/{path}?ref={BRANCH}"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else None

def read_file(file_path):
    url = f"{BASE_URL}/{file_path}?ref={BRANCH}"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return base64.b64decode(response.json()["content"]).decode("utf-8")
    return None

# ===================================================
# Main Daily Runner (Modified for Gemini)
# ===================================================
def extract_number(fname):
    match = re.search(r"chapter(\\d+)_", fname)
    return int(match.group(1)) if match else 0

def main():
    plan_filename = "book_plan.md"
    plan_content = read_file(plan_filename)

    if plan_content is None:
        print("Day 1: Creating book plan and first chapter...")
        crew = Crew(agents=[planning_agent], tasks=[plan_book_task])
        plan_result = crew.kickoff()  # Remove .raw
        plan_text = "\n".join(plan_result) if isinstance(plan_result, list) else plan_result # Handle list or string
        github_commit(REPO_FULL_NAME, plan_text, plan_filename, "Added book plan")
        chapter_titles = [line.split(".", 1)[1].strip() for line in plan_text.splitlines() if "." in line]
    else:
        print("Continuing book writing...")
        chapter_titles = [line.split(".", 1)[1].strip() for line in plan_content.splitlines() if "." in line]

    files = list_files()
    chapter_files = [file["name"] for file in files if file["name"].startswith("chapter") and file["name"].endswith(".md")]
    chapter_files_sorted = sorted(chapter_files, key=extract_number)
    current_chapter_count = len(chapter_files_sorted)

    if current_chapter_count >= 60:
        print("All chapters are written. Process complete.")
        return

    next_chapter_number = current_chapter_count + 1
    next_chapter_title = chapter_titles[next_chapter_number - 1] if next_chapter_number - 1 < len(chapter_titles) else f"Chapter {next_chapter_number}"

    context_text = "\n".join([f"Excerpt from {fname}: " + read_file(fname)[:300] for fname in chapter_files_sorted[-3:]])

    write_chapter_task.description = f"Write Chapter {next_chapter_number}: {next_chapter_title}. Context: {context_text}"
    crew = Crew(agents=[writing_agent], tasks=[write_chapter_task])
    chapter_result = crew.kickoff()
    chapter_content = "\n".join(chapter_result) if isinstance(chapter_result, list) else chapter_result
    # ... (rest of your code) ...
    format_markdown_task.description = f"Format the following chapter content to ensure correct Markdown structure: {chapter_content}"
    crew = Crew(agents=[formatting_agent], tasks=[format_markdown_task])
    formatted_chapter = crew.kickoff()  # Remove .raw, get the string directly
    # If formatted_chapter is a list, join it into a string
    formatted_chapter_content = "\n".join(formatted_chapter) if isinstance(formatted_chapter, list) else formatted_chapter

    chapter_filename = f"chapter{next_chapter_number}_{next_chapter_title.replace(' ', '_')}.md"
    github_commit(REPO_FULL_NAME, formatted_chapter_content, chapter_filename, f"Added {chapter_filename}")
    print(f"Day {next_chapter_number} completed: {chapter_filename} committed.")

if __name__ == "__main__":
    main()

Continuing book writing...




> Entering new CrewAgentExecutor chain...


Thought: I now can give a great answer

Final Answer:

# Chapter 3: It is Mined from Deep Within the Earth and Strictly Controlled by the Ruling Council

The air hung heavy with the scent of ozone and something else, something metallic and faintly acrid, as Elara descended the spiraling staircase. Each step echoed unnervingly in the vast, cylindrical shaft, the only light source emanating from the flickering gas lamps affixed to the walls at irregular intervals. This wasn't the bustling, clockwork heart of Aethelburg she knew above; this was something far older, far more clandestine.

This was the Aetherium Mine.

She'd spent weeks piecing together the whispers, the hushed conversations overheard in taverns and back alleys, the cryptic symbols etched into forgotten monuments. Aetherium. The source of Aethelburg's unique magic, its interwoven technology, its very lifeblood.  It wasn’t steam or gears that truly powered the city; it was this elusive substance, mined from deep within the e